In [10]:
from google.colab import auth
auth.authenticate_user()

# bibliotecas
from google.cloud import storage
from datetime import datetime
import pandas as pd
import io

# cliente de almacenamiento
client = storage.Client()

# bucket y archivo
bucket_name = 'archivos_raw'
file_name = 'MassachusetsGoogle.json'

# obtener el bucket
bucket = client.get_bucket(bucket_name)

# obtener el blob (archivo)
blob = bucket.blob(file_name)

# descargar el contenido del archivo como string
file_content = blob.download_as_string()

# leer el contenido como DataFrame
df = pd.read_json(io.BytesIO(file_content))

# mostrar el DataFrame
df.head()


,user_id,name,time,rating,text,pics,resp,gmap_id
0,1.180340e+20,Donna Carlson,1619732714443,5,People run it make sure you are comfortable,None,None,0x89e4a103dd188585:0xfadc47938337579f
1,1.101673e+20,michael simmons,1615400031636,5,When you need a helping hand.,None,None,0x89e4a103dd188585:0xfadc47938337579f
2,1.122136e+20,Paul Murray,1593992657241,5,Very Helpful and understanding people definite...,None,None,0x89e4a103dd188585:0xfadc47938337579f
3,1.081745e+20,Elba De Oliveira,1582074606394,5,Very good place nice things but on Saturday th...,None,None,0x89e4a103dd188585:0xfadc47938337579f
4,1.089901e+20,Sherri Mayne,1572962045881,5,Awesome place,None,None,0x89e4a103dd188585:0xfadc47938337579f


In [ ]:
df.isnull().sum()

user_id          0
name             0
time             0
rating           0
text       1128070
pics       2330502
resp       2183863
gmap_id          0
dtype: int64

In [12]:
df_new = df.drop(columns=['name','pics','resp'])

# Convertir la columna 'time' de milisegundos a segundos
df_new['time'] = df_new['time'] / 1000

# Convertir el Unix timestamp a una fecha legible
df_new['date'] = pd.to_datetime(df_new['time'], unit='s').dt.date
df_new['platform'] = 'Google'
df_new = df_new.drop(columns=['time'])
df_new.head()


,user_id,rating,text,gmap_id,date,platform
0,1.180340e+20,5,People run it make sure you are comfortable,0x89e4a103dd188585:0xfadc47938337579f,2021-04-29,Google
1,1.101673e+20,5,When you need a helping hand.,0x89e4a103dd188585:0xfadc47938337579f,2021-03-10,Google
2,1.122136e+20,5,Very Helpful and understanding people definite...,0x89e4a103dd188585:0xfadc47938337579f,2020-07-05,Google
3,1.081745e+20,5,Very good place nice things but on Saturday th...,0x89e4a103dd188585:0xfadc47938337579f,2020-02-19,Google
4,1.089901e+20,5,Awesome place,0x89e4a103dd188585:0xfadc47938337579f,2019-11-05,Google


In [ ]:
df_new.isnull().sum()

user_id           0
rating            0
text        1128070
gmap_id           0
platform          0
dtype: int64

In [13]:
duplicate_rows = df_new[df_new.duplicated()]

if len(duplicate_rows) > 0:
  print(f"Hay {len(duplicate_rows)} filas duplicadas en el DataFrame.")
else:
  print("No hay duplicadas")


Hay 19739 filas duplicadas en el DataFrame.


In [14]:
df_new = df_new.drop_duplicates()
df_new.head()

,user_id,rating,text,gmap_id,date,platform
0,1.180340e+20,5,People run it make sure you are comfortable,0x89e4a103dd188585:0xfadc47938337579f,2021-04-29,Google
1,1.101673e+20,5,When you need a helping hand.,0x89e4a103dd188585:0xfadc47938337579f,2021-03-10,Google
2,1.122136e+20,5,Very Helpful and understanding people definite...,0x89e4a103dd188585:0xfadc47938337579f,2020-07-05,Google
3,1.081745e+20,5,Very good place nice things but on Saturday th...,0x89e4a103dd188585:0xfadc47938337579f,2020-02-19,Google
4,1.089901e+20,5,Awesome place,0x89e4a103dd188585:0xfadc47938337579f,2019-11-05,Google


In [15]:
# prompt: exportar la tabla al bucket etl_archivos

# definir el nombre del archivo a exportar
file_name = 'MassachusettsGoogleReviews.csv'

# convertir el DataFrame a CSV
csv_data = df_new.to_csv(index=False)

# crear un nuevo blob en el bucket de destino
blob = client.bucket('etl_archivos').blob(file_name)

# subir el contenido del DataFrame al blob
blob.upload_from_string(csv_data)

# imprimir un mensaje de confirmación
print(f"El archivo '{file_name}' ha sido exportado al bucket 'etl_archivos'.")


El archivo 'MassachusettsGoogleReviews.csv' ha sido exportado al bucket 'etl_archivos'.
